<a href="https://colab.research.google.com/github/gaixen/Frames-to-Fables_assignment01/blob/main/Assignment%205/241027_Soham_Mukherjee_Assignment_05_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jun 24 10:40:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [33]:
import torch

In [16]:
from PIL import Image
IMAGE_URLS = [
    "https://images.unsplash.com/photo-1750126833705-ba98013f16f3",
    "https://images.unsplash.com/photo-1749627995669-4d4dda3a9c1d",
    "https://images.unsplash.com/photo-1744294724362-3f5c404c771a",
    "https://images.unsplash.com/photo-1750075750236-3f8924fc0e35"
]

## pipeline method to generate captions

In [14]:
from transformers import pipeline
captioner = pipeline("image-to-text" , model = "Salesforce/blip-image-captioning-large")
captioner(IMAGE_URLS[0])

Device set to use cuda:0


[{'generated_text': 'there are many boats that are tied to the dock in the water'}]

In [17]:
def image_to_text(image_url):
  image_captions = []
  for image in image_url:
    image_captions.append(captioner(image))

  return image_captions

image_captions = image_to_text(IMAGE_URLS)

def image_to_tensors(i):
  image = Image.open(requests.get(IMAGE_URLS[i] , stream = True).raw)
  inputs = processor(text = [image_captions[i]] , images = image , return_tensors = "pt" , padding = True)
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image
  probs = logits_per_image.softmax(dim = 1)
  return probs

In [20]:
image_captions

[[{'generated_text': 'there are many boats that are tied to the dock in the water'}],
 [{'generated_text': 'there are yellow flowers growing on the side of a mountain'}],
 [{'generated_text': 'someone taking a picture of a tree with a camera'}],
 [{'generated_text': 'there is a blue car parked in front of a building'}]]

## Captioning of the URL images


In [22]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [24]:
captions = []
def image_to_text(i):
  img_url = IMAGE_URLS[i]
  raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
  text = "a photography of"
  inputs = processor(raw_image, text, return_tensors="pt")
  out = model.generate(**inputs)
  print(processor.decode(out[0], skip_special_tokens=True))
  inputs = processor(raw_image, return_tensors="pt")
  out = model.generate(**inputs)
  captions.append(processor.decode(out[0], skip_special_tokens=True))

In [25]:
for i in range(len(IMAGE_URLS)):
  image_to_text(i)

a photography of a row of boats tied to a dock in a lake
a photography of a mountain with a bunch of yellow flowers
a photography of a person taking a picture with a camera
a photography of a blue car parked in front of a building


## CLIP Model based approach

In [28]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

MODEL = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(MODEL)
clip_processor = CLIPProcessor.from_pretrained(MODEL)

In [34]:
tensors = []
def image_to_tensors(i):
  image = Image.open(requests.get(IMAGE_URLS[i] , stream = True).raw)
  inputs = clip_processor(text = [captions[i]] , images = image , return_tensors = "pt" , padding = True)
  with torch.no_grad():
    outputs = clip_model(**inputs)
  image_embeds = outputs.image_embeds
  tensors.append(image_embeds.squeeze(0))

for i in range (len(IMAGE_URLS)):
  image_to_tensors(i)

In [36]:
query = input("Enter which image are you looking for: ")

Enter which image are you looking for: hilly yellow flowers


In [39]:
inputs = clip_processor(text = [query] , images = None , return_tensors = "pt" , padding = True)
with torch.no_grad():
  outputs = clip_model.get_text_features(**inputs)
text_embeds = outputs.squeeze(0)

In [41]:
similarity_scores = [torch.dot(text_embeds, tensor) for tensor in tensors]
most_similar_index = similarity_scores.index(max(similarity_scores))
print(f"The most similar image is: {IMAGE_URLS[most_similar_index]} and the index is : {most_similar_index}")

The most similar image is: https://images.unsplash.com/photo-1749627995669-4d4dda3a9c1d and the index is : 1


## BLIP based Task

In [21]:
import json
import os
# loading the json file
with open("/content/filtered_questions.json", "r") as f:
    qa_raw = json.load(f)

qa = []
for entry in qa_raw:
    question, answer, img_index = entry
    qa.append({
        "question": question,
        "answer": answer,
        "image": f"{img_index}.png"
    })


In [22]:
from PIL import Image
for i in range(2):
    print(f"Q: {qa[i]['question']}")
    print(f"A: {qa[i]['answer']}")

Q: what is the blue shape?
A: rectangle
Q: what color is the shape?
A: blue


In [23]:
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

def vqa(question, image_path):
    raw_image = Image.open(image_path).convert("RGB")
    inputs = processor(raw_image, question, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(**inputs)

    return processor.decode(out[0], skip_special_tokens=True)


In [25]:
import pandas as pd
results = []
for pair in qa:
    img_path = os.path.join("images", pair["image"])
    pred = vqa(pair["question"] , img_path)
    results.append({
        "image": pair["image"],
        "question": pair["question"],
        "ground_truth": pair["answer"],
        "predicted": pred
    })
df = pd.DataFrame(results)
df.to_csv("results.csv", index=False)